# Defining a System Mueller Matrix

Models are stored in a "system Mueller matrix" object from the `pyMuellerMat` package (author Max Millar-Blanchaer). See the pyMullerMat GitHub for more information on functionality: https://github.com/maxwellmb/pyMuellerMat. 

# Step 1: Figure Out What Common Mueller Matrices You Need

Your system Mueller matrix will be defined based on a chain of "common Mueller matrix" objects that are stored in the source code of pyMuellerMat. Browse the common Mueller matrix functions and note the names of the functions (you will need it later): https://github.com/maxwellmb/pyMuellerMat/blob/physical_models/pyMuellerMat/common_mm_functions.py. 

# Step 2: Define a System Dictionary

This is where all the information necessary to generate the system Mueller matrix is stored. It is a dictionary of dictionaries. First you add a `"components"` key, then you add all of your components as dictionaries inside this dictionary. You can name components whatever you'd like. For `"type"` you put the name of the `common_mm_function` you found in the last step. For `properties`, add arguments for the `common_mm_function`. Along with whatever unique arguments each `common_mm_function` has, you can pass `theta` and/or `delta_theta` to rotate each component. I use `theta` for known rotation angles and `delta_theta` for fitting offset angles. Finally, at the end of each component, add an internal tag: `"tag":"internal"`.

NOTE: All retardances `phi` are in radians and rotation angles `theta, delta_theta` are in degrees.

In [2]:
# Example system dictionary, all params set to zero

system_dict = {
        "components" : {

            "wollaston" : {
            "type" : "wollaston_prism_function",
            "properties" : {"beam": 'o','eta':0}, 
            "tag": "internal",
            },

            "image_rotator" : {
            "type" : "elliptical_retarder_function",
            "properties" : {"phi_45":0,"phi_h": 0, "phi_r": 0, "theta": 0, "delta_theta":0},
            "tag": "internal",
            },
            
            "hwp" : {
                "type" : "general_retarder_function",
                "properties" : {"phi": 0, "theta": 0, "delta_theta": 0},
                "tag": "internal",
            },


            "calibration_polarizer" :{
                "type" : "diattenuator_retarder_function",
                "properties" : {"epsilon": 0, "delta_theta": 0},
                "tag": "internal",
            },
}
    }

# Step 4: Generate a System Mueller Matrix

In [9]:
from pyPolCal.utils import generate_system_mueller_matrix

system_mm = generate_system_mueller_matrix(system_dict)

# You can evaluate the system Mueller matrix at this point

system_mm.evaluate() 

array([[0.5, 0. , 0. , 0. ],
       [0. , 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

# Optional: Custom Common Mueller Matrices

If you would like to add a Mueller matrix to your chain that isn't available in the default script or you would like a Mueller matrix with hardcoded values as a function of some parameter that isn't diattenuation, retardance, or rotation angle, you'll have to modify the source code of `pyMuellerMat`.

To do this, you must first clone the correct branch of pyMuellerMat onto your computer. In your terminal:

Then, navigate to `pyMuellerMat` and install in editable mode:

In python, open the script `pyMuellerMat/common_mm_functions.py` and familiarize yourself with the format of these functions. Make your own custom Mueller matrix function. Note: you MUST have a default value for all arguments in your function! Otherwise, the code will break.

Once you finish your function, go to the script `pyMuellerMat/common_mms.py`. Here you will initialize your object. It will have this form:

In [ ]:
class ArbitraryMatrix(MuellerMat.MuellerMatrix):
    def __init__(self, name='ArbitraryMatrix', matrix_function=arbitrary_matrix_function):
        """
        Initialize an ArbitraryMatrix object.

        Parameters:
        - name (str): The name of the matrix.
        - matrix_function (callable): A function that returns a 4x4 matrix. Defaults to `arbitrary_matrix_function`.
        """
        super(ArbitraryMatrix, self).__init__(matrix_function, name=name)

This is very useful for doing global fits if you have an IFS instrument! For CHARIS, I hardcoded fitted retardance values of the derotator as a function of wavelength so I could fit global parameters for the HWP retardance. You can see these in the "Special" category at the very bottom of `pyMuellerMat/common_mm_functions.py`.